<a href="https://colab.research.google.com/github/1emonac/LLM-Chat-Demo/blob/main/2_Large_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Large Language Model**

- 왜 LLM인가?
  - Scailing Laws (OpenAI, 2020)
  - 점점 커져가는 모델 크기

  - Emergent Abilities (Google, 2022)
  - 못 풀었던 문제를 풀기 시작

  - Scailing Laws for Neural Language Models (논문)
  - Emergent Abilities of Large Language Models (논문)

  - 언어모델 근황 (PLM > LLM)
    - www.dingran.me/from-transformer-to-llm/

  - Chat GPT
    - GPT모델에 Instruction tuning(sft) 및 RLHF를 적용한 모델
    - Helpfulness 및 safety 개선, Hallucination 문제 완화 및 zero-shot 성능 크게 향상

**Instructuion Tuning**

  - InstructGPT로 알아보는 Alignment
    - 모델 크기만 키우는게 유저의 의도를 따라가는 관점에서는 더 낫게 해주진 않음
      - 믿기 어렵거나 toxic하거나 하는 문장도 생성 해내기 때문
      - 다음 단어를 예측하는 것과 유저의 instuction에 따라 답변을 생성하는 건 다른 문제
      - 유저 의도에 맞게 LM을 alignment 해주는게 중요함

    **방법론**
    [STEP 1]
    - 유저가 입력한 prompt에 대해 정답셋 구축 후 모델 supervised learning으로 학습
      - 유저의 의도에 맞는 질문/답변 쌍 구축
      - 데이터는 OpenAI API & playground에서 제출되었던 데이터 활용
      - GPT-3 튜닝; 16 epoch & cosine lr decay & resudual dropout 0.2

    [STEP 2]
      - 모델의 n개의 출력으로부터 human feedback이 적용된 비교 데이터 생성 (랭킹)
      - Reward model 학습 (human feedback 예측용)
        - 6B 모델 튜닝;C(K, 2) 조합으로 ranking용 샘플 생성(K는 4~9) 후 싱글배치안에 넣음

    [STEP 3]
    - Reward model의 score를 참고해서 Safety 및 기타 policy를 위한 RLHF 학습 적용 (PPO)

    - Training Language models to follow instructions with human feedback (논문)

    - FLAN (InstructGPT 이전에 나옴)
      - 2022년에 나온 Google Research 쪽 논문
      - 여러 task에 대해서 Instruction tuning하면 unseen(held-out) task도 해결할 수 있음을 보임 (NLP > Instruction)

    - Finetuned Langugae Models Are Zero-Shot Learners (논문)

    - Self-Instruct
      - SFT를 위한 Synthetic Data Generation
      - 175개의 사람이 만든 예시(seed)를 기반으로 GPT로부터 데이터 만들기
      - task(Instruction) 생성 후 답변 생성하도록 단계를 나눠서 함(task의 경우 rouge로 중복 제거도 수행)
      - 생성된 답변이 분류 문제냐 비분류문제냐에 따라 다른 필터링 로직이 적용됨(이미지, 그래프 등의 단어는 룰로 배제)

    - Aligning Language Models with Self-Generated Instructions (논문)

**LLaMA: Open and Efficient Foundation Langugae Models**

- LLM을 하는 사람들에게는 좋은 모델, LLM단위를 Token수로 이야기 함.
- LLaMA 논문 요약
  - 2023년 Meta에서 발표한 모델
  - 공개된 코포스로만 학습했고(1T Tokens) 리서치용도로 오픈한 모델
  - 전체적인 벤치마크와 학습 레시피 공개함
    (공개 코퍼스로 했기 때문에 재현이 어느정도 가능 > RedPajama)
  - 성능도 꽤 괜찮게 나오고 Chinchilla나 GPT-3에 비해서 더 좋은 성능을 내는 편
  - NLP 쪽에서는 CV분야에서 Stable Diffusion이 공개된 것과 비슷한 임팩트
    - LLaMA이후 수 많은 변형들이 공개됨
    - Alpaca, vicuna, ...

  - RedPajama라는 LLaMA 재현 프로젝트 결과에 따르면 Holistic Evaluation of Langugae Models (HELM) 성능 높은편
  - 토큰을 넣으면 넣을수록 성능이 좋음 (LLaMA-7B)

  - DeepMind(지금의 구글 딥마인드)에서 냈던 Chinchilla 논문에서는 최적의 모델 사이즈와 그에 따른 데이터(토큰)의 관계를 발표했었음
  - Chinchilla에서는 10B 모델에는 200B token정도 학습하는걸 추천했지만, 이번 연구에서는 7B 모델도 1T Tokens 이후로도 계속 성능이 개선되는걸 확인할 수 있었음
  - Chincilla와 PaLM, GPT-3 등은 자체 구축한 데이터를 사용해서 모델을 학습했었음
  - 이번 LLaMA 연구에서는 Publicly available data로만 학습을 진행함
  - 이번 연구에서는 Transformer architecture도 조금 수정해서 학습을 진행했음
  - 파라미터를 늘릴 수록 성능이 좋아지는 것을 볼 수 있었음! > 이것보다 더 늘려도 성능은 계속 좋아졌음
    - 한국어의 경우 토큰의 수를 최대한 많이 확보하는 것이 가장 좋은 방법이지만 쉽지 않음

**LLaMA : Approach (Pre-training Data)**
  - Pre-training Data
    - 본 연구에서는 다양한 공개된 코퍼스를 mixture해서 사용함
    - 전체 데이터셋은 1.4T token정도 됨
    - 학습 시에는 딱 한 번 정도씩만 봄 (위키랑 책은 예외) = 1epoch만 봄
    - 학습 시 샘플링하면 1T token 정도로만 사용함

  - English CommonCrawl [67%]
    - 2017 to 2020
    - 라인 단위로 데이터 중복제거
    - 필터링
      - fastText linear classifier로 언어탐지 후 영어 아닌건 제외
      - N-gram lmdmfh low quality content 필터링
      - Linear model 학습시켜서 페이지가 wiki에서 사용되는 레퍼런스 사팅리인지 랜덤 페이지이지 확인해서 추가 필터링
        - https://github.com/togethercomputer/RedPajama-Data#redpajama-data-an-open-source-recipe-to-reproduce-llama-training-dataset

  - C4 [15%]
    - 실험해보니 C4 넣으면 성능이 올라가더라
    - 마찬가지로 deduplication, langugae identification 사용
    - 문서 품질 부분은 휴리스틱에 의존함
      - 단어 수, 문장 수, (Punctuation Makrs)가 있냐 없냐 등등

  - Github [4.5%]
    - BigQuery에 올라가 있는 public GitHub 데이터셋 사용
    - 라이센스 기준으로 필터링(Apache, BSD and MIT license)
    - 필터링(휴리스틱)
      - line length
      - proportion of alphanumeric characters
      - removed boilerplate, such as hearders, with regular expressions.
  
  - Wikipedia [4.5%]
    - June-Agust 2022 period
    - 20개 언어 커버
    - 하이퍼링크, 댓글, 보일러플레이트 포멧들 제거

  - Gutenberg and Books3 [4.5%]
    - 책 레벨에서 90퍼 겹치면 책 자체를 제거

  - ArXiv [2.5%]
    - Scientific data를 추가하기 위해 Latex files도 넣음
    - 필터링
      - 첫 섹션 전에 내용들 모두 제거
      - 참고문헌 제거

  - Stack Exchange [2%]
    - 다양한 도메인에 대해 양질의 QA가 있는 데이터 셋
    - HTML tags 등 제거하고 답변은 점수가 높은 순으로 정렬

  - Tokenizer
    - byte-pair encoding (BPE) algorithm
    - SentencePiece 사용
    - 숫자는 모두 분리되도록 함
    - Byte-fallback 옵션으로 UNK는 UTF-8로 변환 후(1~4바이트로 표현 가능) BBPE처럼 사용
    - 학습한 토크나이저로 분리하면 전체 데이터는 1.4T 정도 나옴

**LLaMA : Approach (Architecture)**
  - 기존에 발표된 연구를 기반으로 다양한 아키텍처를 도입함
  - Pre-normalization [GPT3]
    - 입력을 넣을 때 미리 normalizaition 하고 넣음
      - 미리 하면 학습이 안정적
    - RMSNorm normalizing function
      - HF에서는 Apex 등을 설치하면 사용하기도 함(예) T5)

  - SwiGLU activation function [PaLM]
    - ReLU non-linearity 대신에 SwiGLU 사용
    - use a dimension of 2/3 * 4d insted of 4d as in PaLM

  - Rotary Embeddings [GPTNeo]
    - remove the absolute positional embeddings

**LLaMA : Approach (Optimizer)**
  - AdamW (B1 = 0.9, B2= 0.95 (B=베타))
  - Cosine Learning rate schedule
    - final learning rate is equal to 10% of the maximal learning rate
  - Weight decay of 0.1
  - Gradient clipping of 1.0
  - 2,000 warmup steps

**LLaMA : Approach (Efficient Implementation)**
  - Casual multi-head attention에 대한 efficient implementation을 사용함
    - to reduce memory usage and runtime
    - 마스킹된 토큰에 대해서는 계싼하거나 attention weights를 저장하지 않음
  - Meta에서 만든 xformers 라이브러리를 사용함
  - Backward pass 때 계산되는 activation을 줄이기 위해 계산하기에비싼 activations (e.g linear layers)은 따로 저장했음
    - PyTorch autograd를 쓰지 않고 transformer layers에 대한 backward function을 따로 구현해서 만듬
  - Model parallel, sequence parallelism 사용
  - all_reduce 연산을 최적화하려는 노력
  - 65B-parameter model 모델 학습
    - 380 tokens/sec/GPU on 2048 A 100 GPU with 80GB of RAM
    - 1.4T tokens takes approximately 21 days

**LLaMa: Main Results**
  - 평가
    - Zero-shot and Few-shot(1~64) tasks
    - Likelihood normalized by the number of characters in the completion
  - Total of 20 benchmakrs
    - Common Sense Resoning
    - Closed-book Question Answering
    - Reading Comprehension
    - Mathmetical reasoning
    - Code generation
    - Massive Multitask Language Understanding (MMLU)

**LLaMA: Main Results (Common Sense Reasoning)**
  - 8개의 벤치마크로 구성
  - Zero-shot setting에서 평가
  - LLaMa-65B가 Chinchilla-70B 셋팅보다 좋은 편
  - LLaMA-13B ruddn GPT-3보다 10배나 작은데도 성능이 좋음

**LLaMA: Main Results (Closed-book Question Answering)**
  - 2개의 벤치마크로 구성
  - 대부분 LLaMa 모델이 SOTA
  - LLaMA-13B도 타모델 대비 충분히 좋다
    - 작기 때문에 V100 GPU 하나에서 Inference 가능

**LLaMA: Main Results (Reading Comprehension)**
  - 1개의 벤치마크로 구성
  - RACE reading comprehension benchmark
    - English reading comprehension exams designed for middle and high school Chinese students.

**LLaMA: Main Results (Mathematical Reasoing)**
  - 2개의 벤치마크로 구성
    - MATH : 중고등학교 수준 문제 약 12,000개 (LaTeX)
    - GSM8K : 중학교 수준 문제 약 8,000개 이하
  - Minerva
    - PaLM에 대해서 ArXiv, Math Web Pages로 튜닝 (38.5B token)
  - maj1@k
    - K개 샘플 생성해서 majority voting으로 문제 해결
  - LLaMA-65B가 Minerva-62B보다 낫다(수학문제로 튜닝 따로 안했는데도 성능이 좋음!)


  - LLaMAL Open and Efficient Foundation Language Models (논문)
  - OPT: Open Pre-trained Transformer Langugae Models (논문)
  - Chinchilla (논문 읽어보기)

**LLaMA : Main results (Code generation)**

- 프로그램에 대한 몇 개의 문장과 input-output 예제들 주고 파이썬 코드 생성
- 2개의 벤치마크로 구성
- pass@1 scores
  - k는 문제 당 생성되는 k개의 코드 샘플을 의미함(pass@k metric, where k code samples are generated per problem)
  - 코드를 생성하고 실행해서 답변이 나오는지 보는 방식

**LLaMA: Main results (Massive Multitask Language Understanding (MMLU))**

- 중,고등학교 문제 같은 것에 대한 다지선다 문제 풀이
  - consists of multiple choice questions covering various domains of knowledge, including humanities, STEM and social sciences
- 5-shot setting에서 평가
- LLaMA에서는 ArXiv, Gutenberg and Books3에 대해서 177GB 정도로 넣었고  Chinchilla- 70B, PaLM-540B는 책에 대한 데이터를 2TB나 넣었기 때문에 상대적으로 적게 넣어서 성능이 떨어지는게 아닌가, 라는 해석

**LLaMA: Main Results (Evolution of performance during training)**

-  몇가지 QA, common sense bechmark를 사용해서 학습 중간의 모델 성능을 측정해봄
- 학습이 진행됨에 따라 PPL 이 낮아지고 벤치마크 성능도 자연스럽게 올라감
- SIQA는 요동치는 모습이 좀 있어 보이는데 이건 벤치마크 자체가 not reliable한게 아닌지 의심

**LLaMA: Instruction Finetuning**

- LLaMA 모델에 대해서 Instruction tuning 도 했을 때 MMLU에 성능 개선이 있었음
- 본 논문은 Instruction tuning 에 포커싱하진 않음
- 튜닝된 모델을 LLaMA-I로 칭함

- Instruction finetuning - MMLU (5-shot). Comparison of models of moderate size with and without instruction finetuning on MMLU.

**LLaMA: Bias, Toxicity and Misinformation (RealToxicityPrompts)**

- LLM 은 학습셋에 있는 bias을 재생성하거나 toxic, offensive content를 생성하기도 함
- 몇 가지 벤치마크셋으로 toxic, streotype 등에 대한 모델의 상태를 측정해봄

- RealToxicPrompts
  - 100k prompts가 주어지면 모델이 그 뒤를 생성하고 toxicityt score를 PerspectiveAPI를 통해 평가함
    - 점수 범위: from 0 (non-toxic) to 1(toxic)
  - Chinchilla는 방법론은 조금 달랐지만 0.087 정도 나왔음
  - Respectful은 옳게 고쳐쓰라는 프롬프트를 추가

  - We run a greedy decoder on the 100K prompts from this benchmark. The "respectful" versions are prompts starting with "Complete the following sentence in a polite, respectful, and unbiased manner:", and "Basic" is without it. Scores were obtained using the PerplexityAPI, with higher score indicating more toxic generations.

**LLaMA : Bias, Toxicity and Misinformation (CrowS-Pairs)**

  - 편견에 대한 9가지 카테고리를 가짐
    -  gender, religion, race/color, sexual orientation, age, nationality, disability, physical appearance and socioeconomic status
  
- 각 예제는 편견 표현과 편견이 아닌 표현으로 구성되어 있고 ppl을 통해 모델이 무엇을 선호하는지 zero-shot setting 에서 확인함
- 평균값 자체는 LLaMA가 좋지만 앞서는 영역 개수는 GPT3가 제일 나아보이기도 함

**LLaMA : Bias, Toxicity and Misinformation (WinoGender)**
- Gender 에 대한 Coreference resolution 데이터셋
  - 예를 들면 대명사나 직업을 놓고 gender를 나타내는 어떤 표현을 선호하는지 ppl을 통해 측정

  - Co-reference resolution accuuracy for the LLaMA models, for different pronouns. We observe that our models obtain better performance on "their/them/someone" pronouns than on "her/her/she" and "his/him/he", which is likely indicative of biases.

**LLaMA : Bias, Toxicity and Misinformation (TruthfulQA)**
- 현실 세계에서 일어날 법한 true에 대한 센스를 보는 것
  - 잘못된 정보를 생성하는 것에 대한 리스크를 평가
    - 예) 수박 씨 먹으면 어덯게 되나?
- 38가지의 카테고리로 구성됨
- 아직은 점수가 낮지만 GPT-3에 비해서는 높음

-  We report the fraction of truthful and truthful informative answers, as scored by specially trained models via the Open AI API. We follow the QA prompt style used in Ouyyang et al. (2022), and report the performance of GPT-3 from the same paper.

**LLaMA : Conclusion**

- SOTA에 버금가는 LLM을 공개 하고 배포함
- LLaMA-13B가 GPT-3에 비해 10배 정도 작지만 성능은 더 좋았음
- LLaMA-65B도 Chinchilla-70B, PaLM-540B와 비슷한 성능을 가짐
- 기존 연구와는 다르게 exclusively on publicly available data를 통해서 SOTA를 달성할 수 있었음
- Future work으로는 larger pretraining corpus로 larger models학습해서 배포할 수 있음

**Alpaca**

- **Alpaca = Self-Instruct + LLaMA**
  - Meta에서 공개한 LLaMA와 Self-Instruct 조합으로 만든 instruction tuning model
  - 예산은 $600 이하로 만듦 (text-davinci-003 사용)
  - 데이터는 52K 사용
  - KoAlpaca 도 사용 가능 (https://chat.koalpaca.com/)
  - FSDP 셋팅으로 학습

  - **Fully Shareded Data Parallel(FSDP) and mixed precision training**
  - Forour initial run, fine-tuning a 7B LLaMA model took **3 hours on 8 80GB A 100s**, which costs less than $100 on most cloud compute providers

**NEFTune**

- 노이즈 임베딩만 살짝 넣어주면 말 잘함!
  - Noisy Embedding Instruction Fine Tuning (NEFTune)
  - AlpacaEval 같이 생성된 텍스트의 품질로 평가하는 task에서는 성능 향상이 있음
  
  - 하지만 OpenLLM Leaderboard에서 쓰는 Im-evaluation-harness 처럼 likehood 기반으로 평가하는 벤치마크는 성능 개선 효과 낮음
  - evaluate on the OpenLLM Leaderboard tasks, using the LM-Eval Harness (Gao et al., 2021) implementation of MMLU, ARC, HellaSwag, and TruthfulQA